In [11]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [12]:
from langchain_teddynote.tools.tavily import TavilySearch

In [13]:
tool = TavilySearch(max_results=3)

In [14]:
tool.invoke({"query": "테디노트 랭체인 튜토리얼"})

'["<document><title> - LangChain 한국어 튜토리얼 - WikiDocs</title><url>https://wikidocs.net/book/14314</url><content> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기 01. 설치 영상보고 따라하기 02. OpenAI API 키 발급 및 테스트 03. LangSmith 추적 설정 04. OpenAI API 사용(GPT-4o 멀티모달) 05.</content><raw><랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 - WikiDocs\\n<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기 01. 설치 영상보고 따라하기 02. OpenAI API 키 발급 및 테스트 03. LangSmith 추적 설정 04. OpenAI API 사용(GPT-4o 멀티모달) 05. LangChain Expression Language(LCEL) 06. LCEL 인터페이스 07. Runnable CH02 프롬프트(Prompt) 01. 프롬프트(Prompt) 02. 퓨샷 프롬프트(FewShotPromptTemplate) 03. LangChain Hub 04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers) 01. Pydantic 출력 파서(PydanticOutputParser) 02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser) 03. 구조화된 출력 파서(StructuredOuputParser) 04. JSON 출력 파서(JsonOutputParser) 05. 데이터프레임 출력 파서(PandasDataFrameOutputParser) 06. 날짜 형식 출력 파서(DatetimeOutputParser) 07. 열거형 출력 파서(EnumOutputParser) 08. 출력 수정 파서(OutputFixingParser) CH04 모델

In [15]:
# 도구 목록에 추가
tools = [tool]

In [16]:
# 도구 실행
print(tool.invoke("테디노트 랭체인 튜토리얼"))

["<document><title> - LangChain 한국어 튜토리얼 - WikiDocs</title><url>https://wikidocs.net/book/14314</url><content> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기 01. 설치 영상보고 따라하기 02. OpenAI API 키 발급 및 테스트 03. LangSmith 추적 설정 04. OpenAI API 사용(GPT-4o 멀티모달) 05.</content><raw><랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 - WikiDocs\n<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기 01. 설치 영상보고 따라하기 02. OpenAI API 키 발급 및 테스트 03. LangSmith 추적 설정 04. OpenAI API 사용(GPT-4o 멀티모달) 05. LangChain Expression Language(LCEL) 06. LCEL 인터페이스 07. Runnable CH02 프롬프트(Prompt) 01. 프롬프트(Prompt) 02. 퓨샷 프롬프트(FewShotPromptTemplate) 03. LangChain Hub 04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers) 01. Pydantic 출력 파서(PydanticOutputParser) 02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser) 03. 구조화된 출력 파서(StructuredOuputParser) 04. JSON 출력 파서(JsonOutputParser) 05. 데이터프레임 출력 파서(PandasDataFrameOutputParser) 06. 날짜 형식 출력 파서(DatetimeOutputParser) 07. 열거형 출력 파서(EnumOutputParser) 08. 출력 수정 파서(OutputFixingParser) CH04 모델(M

In [17]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

In [18]:
# State 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [19]:
from langchain_openai import ChatOpenAI

In [20]:
# LLM 정의
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [21]:
# LLM 에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [22]:
# 노드 함수 정의
def chatbot(state: State):
    answer = llm_with_tools.invoke(state["messages"])
        
    return {"messages": [answer]}  # 자동으로 add_messages 적용

In [23]:
from langgraph.graph import StateGraph

In [24]:
# 상태 그래프 초기화
graph_builder = StateGraph(State)

In [25]:
# 노드 추가
graph_builder.add_node("chatbot", chatbot)

In [26]:
import json
from langchain_core.messages import ToolMessage

In [27]:
class BasicToolNode:
    """Run tools requested in the last AIMessage node"""

    def __init__(self, tools: list) -> None:
        # 도구 리스트
        self.tools_list = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        # 메시지가 존재할 경우 가장 최근 메시지 1개 추출
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")

        # 도구 호출 결과
        outputs = []
        for tool_call in message.tool_calls:
            # 도구 호출 후 결과 저장
            tool_result = self.tools_list[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                # 도구 호출 결과를 메시지로 저장
                ToolMessage(
                    content=json.dumps(
                        tool_result, ensure_ascii=False
                    ),  # 도구 호출 결과를 문자열로 변환
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )

        return {"messages": outputs}

In [28]:
# 도구 노드 생성
tool_node = BasicToolNode(tools=[tool])

In [29]:
# 그래프에 도구 노드 추가
graph_builder.add_node("tools", tool_node)